# Conv3d refinements and explainability

In [1]:
import os
import cv2
import numpy as np
import datetime as dt
import shutil
import pandas as pd
import random
from itertools import product
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps

import tensorflow as tf

from keras.layers import *
from keras.models import Sequential
from keras.utils import plot_model, to_categorical
from keras.models import load_model
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score


In [2]:
# List files and ignore .DS_Store if on a Mac
def list_files(directory):
    visible_files = []
    for file in os.listdir(directory):
        if not file.startswith('.'):
            visible_files.append(file)

    return visible_files


# Convert video to frames
def video_to_frames(video_path, img_size, sequence_length):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frames.append(frame)
        if len(frames) == sequence_length:
            break
    cap.release()

    if len(frames) < sequence_length:
        return None  # Ignore short videos

    return np.array(frames)


# Load and Preprocess Videos
def load_videos_from_folders(folder_path, img_size=(64, 64), sequence_length=30):
    # classes = os.listdir(folder_path)
    classes = list_files(folder_path)
    data, labels = [], []

    for label, activity in enumerate(classes):
        activity_folder = os.path.join(folder_path, activity)
        # for video_file in os.listdir(activity_folder):
        for video_file in list_files(activity_folder):
            video_path = os.path.join(activity_folder, video_file)
            frames = video_to_frames(video_path, img_size, sequence_length)
            if frames is not None:
                data.append(frames)
                labels.append(label)

    data = np.array(data)
    labels = to_categorical(labels, num_classes=len(classes))

    return data, labels, classes


# List class names from labeled folders
def get_classes(directory):
    # Get Names of all classes
    classes = list_files(directory)
    # classes = enumerate(classes)

    return classes


# Predict all videos in 
def predict_all3d(model, data, idx, labels, classes):

    # Interpreting the data structure
    predict_idx = idx
    selected_data = data[predict_idx]
    selected_labels = labels[predict_idx]
    true_label = np.argmax(selected_labels, axis=0)
    true_class = classes[true_label]

    # Add extra dimension
    reshaped_data = tf.expand_dims(selected_data, axis=0)

    test_predictions = model.predict(reshaped_data)

    predicted_label = np.argmax(test_predictions, axis=1)
    predicted_class = classes[predicted_label[0]]

    print(f'Predicted class: {predicted_class}\tTrue class: {true_class}')

    if predicted_label == true_label:
        correct = True
    else:
        correct = False

    return predicted_label, predicted_class, true_label, true_class, correct


# Select a random video
def select_random(dir):
    # Get classes
    classes = get_classes(dir)

    # Select random class
    random_class = random.choice(classes)

    # Get all files in random class directory
    path = f'{dir}/{random_class}'
    files = list_files(path)

    # Select random video
    random_video = random.choice(files)
    random_path = f'{path}/{random_video}'

    return random_class, random_path


# Predict activity for one video
def predict_one3d(model, dir, random=False, **kwargs):

    classes = get_classes(dir)

    if random:
        classname, path = select_random(dir)

        # Get random video for a specified class
        if 'classname' in kwargs:
            classname = kwargs.get("classname")
        
    else:
        classname = kwargs.get("classname")
        file = kwargs.get("file")
        path = f"../downloads/r2_test/{classname}/{file}"

    print(f'Predicting {path}\n')

    frames = video_to_frames(path, img_size=(64, 64), sequence_length=30)
    frames = np.array([frames])

    test_predictions = model.predict(frames)

    predicted_label = np.argmax(test_predictions, axis=1)
    predicted_label = predicted_label[0]
    predicted_class = classes[predicted_label]

    print(f'Predicted label: {predicted_label}\tPredicted class: {predicted_class}\tTrue class: {classname}')

    return predicted_label, predicted_class, classname


## Access data, labels, and classes in the dataset

In [3]:
# Get data from the directory
test_data_folder = "../downloads/r2_test/"
data, labels, classes = load_videos_from_folders(test_data_folder)

### Accessing static frames from data

In [128]:
# Number of sequences 
total_sequences = len(data)
sequences_per_class = total_sequences / 10
one_video= data[0]
sequence_len = len(one_video)

# 2nd arg in conv_filters param
one_frame = one_video[0][0]
frame_size = len(one_frame)  # 64x64 image

# I think each row is the max pooled result of the kernel stride for the image
max_pooled_kernel = one_frame[0]  # these are RGB values 
pooled_kernel_size = len(max_pooled_kernel)  # result = 3, kernel is 3x3x3, so it pools 27 pixels

print(f"Num sequences: {total_sequences}") 
print(f"Sequences per class: {sequences_per_class}") 
print(f"Sequence len: {sequence_len}")
print(f"Frame size: {frame_size}")
print(f"Max pooled kernel size: {pooled_kernel_size}")

Num sequences: 3344
Sequences per class: 334.4
Sequence len: 30
Frame size: 64
Max pooled kernel size: 3


## Load a trained model

In [4]:
# Load model
model_path = '../models/2024-11-05-02-15-08-conv3d-model.keras'
model = load_model(model_path)

## Predict one video

In [5]:
# Predict a random video from the test set
directory = "../downloads/r2_test"

predicted_label, predicted_class, classname = predict_one3d(model, directory, random=True)

Predicting ../downloads/r2_test/Therapy/7393448302219676817_r6.mp4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted label: 8	Predicted class: Lying in bed	True class: Therapy


In [ ]:
# Predict a random video from specified class
directory = "../downloads/r2_test"
specified_class = "Resting"

predicted_label, predicted_class, classname = predict_one3d(model=model, 
                                                            dir = directory, 
                                                            random = True,
                                                            classname = specified_class)

Predicting ../downloads/r2_test/Sitting in wheelchair/7393696482609908881_r2.mp4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted label: 6	Predicted class: Talking on phone	True class: Resting


In [269]:
# Predict a specified video from a directory
directory = "../downloads/r2_test"
classname = "Resting"
specified_vid = "7393499193287167121_r7.mp4"

predicted_label, predicted_class, classname = predict_one3d(model=model, 
                                                            dir = directory, 
                                                            random = False,
                                                            classname = classname, 
                                                            file = specified_vid)


Predicting ../downloads/r2_test/Resting/7393499193287167121_r7.mp4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted label: 5	Predicted class: Nurse-Doctor visit	True class: Resting


# Predict entire test set

In [ ]:
# Track correct predictions
correct_predictions = 0
incorrect_predictions = 0

idx = 0

predicted_labels = []
predicted_classes = []
true_labels = []
true_classes = []

# Iterate though test directory
while idx < len(data):

    pl, pc, tl, tc, correct = predict_all3d(model, idx)

    predicted_labels.append(pl)
    predicted_classes.append(pc)
    true_labels.append(tl)
    true_classes.append(tc)

    if correct:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

    idx += 1

print(f'Correct: {correct_predictions}\tIncorrect: {incorrect_predictions}')

# Save data
results_df = pd.DataFrame({"Predicted label": predicted_labels, 
                           "Predicted class": predicted_classes,
                           "True label": true_labels, 
                           "True class": true_classes})

date_time_format = '%Y-%m-%d-%H-%M-%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

file = f'../data/{current_date_time_string}-conv3d-testing.csv'

results_df.to_csv(file, index=False) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 